<a href="https://colab.research.google.com/github/ycyao216/Curry_shot_prediction/blob/main/CurryShotPredDecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests
import tensorflow_decision_forests as tfdf
import pandas as pd

In [ ]:
# features = pd.read_csv('features.csv')
# labels = pd.read_csv('labels.csv')
# df = pd.merge(features, labels, how = 'inner', on = 'Key)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Initalize variables
data_path = 'shot_logs.csv'
player = 'stephen curry'
scaling_features = ['SHOT_DIST', 'SHOT_CLOCK', 'PERIOD', 'TOUCH_TIME', 'CLOSE_DEF_DIST', 'LOCATION', 'FINAL_MARGIN', ]
# , 'SHOT_CLOCK', 'PERIOD', 'TOUCH_TIME', 'CLOSE_DEF_DIST'
train_path = 'train.csv'
valid_path = 'valid.csv'
test_path = 'test.csv'

def process_data(data_path, player, scaling_features, train_path, valid_path, test_path):

    # Load data and select player
    df = pd.read_csv(data_path)
    df = df[df['player_name'] == player]

    # Select features and scale
    for col in scaling_features:
        df[col] = (df[col] - df[col].mean()) / df[col].std()

    # Encode labels to Missed = 0 and Made = 1
    df['SHOT_RESULT'] = df['SHOT_RESULT'].replace({'missed': 0, 'made': 1})

    # Split to train, validation, and test data with a 80/10/10 split
    train, test = train_test_split(df, test_size=0.1, random_state=42)
    train, valid = train_test_split(train, test_size=1/9, random_state=42)

    # Select columns for output
    cols = scaling_features
    cols.append('SHOT_RESULT')

    # Export to csv
    train[cols].to_csv(train_path, index = False)
    valid[cols].to_csv(valid_path, index = False)
    test[cols].to_csv(test_path, index = False)

process_data(data_path, player, scaling_features, train_path, valid_path, test_path)

In [ ]:
train_df = pd.read_csv("train.csv")
print(train_df)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="SHOT_RESULT")

     SHOT_DIST  SHOT_CLOCK    PERIOD  TOUCH_TIME  CLOSE_DEF_DIST  SHOT_RESULT
0    -0.396505   -0.752837 -0.314333    2.568310       -0.700846            1
1     0.716165    0.810186 -0.314333   -0.909482        0.323790            1
2    -0.169650   -0.197552  0.560019    0.113398        0.872703            0
3    -0.094032   -0.115288 -1.188685   -0.909482       -0.481281            1
4     0.651349    0.625091  0.560019   -0.851031        1.311833            1
..         ...         ...       ...         ...             ...          ...
769  -1.433557    0.419430  0.560019    0.931702       -0.078746            1
770   0.921414    1.344904 -0.314333   -0.734131        6.325235            0
771  -0.223663    0.152071  1.434371    0.288749        0.360385            0
772   0.770178   -0.485478 -1.188685   -0.880256        0.396979            1
773  -0.158848   -0.670572  1.434371   -0.529555        0.433573            1

[774 rows x 6 columns]


In [ ]:
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

13/13 [==============================] - 1s 54ms/step


In [ ]:
test_df = pd.read_csv("test.csv")
print(test_df)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="SHOT_RESULT")

    SHOT_DIST  SHOT_CLOCK    PERIOD  TOUCH_TIME  CLOSE_DEF_DIST  SHOT_RESULT
0   -0.958242    0.357732  0.560019    0.025723       -0.774035            0
1    0.813388    0.542827  0.560019    0.610226       -0.261716            1
2    0.759375   -1.616612  1.434371   -0.909482        1.677775            0
3   -1.217505    1.735659  0.560019   -0.851031        0.177414            0
4    0.910612    1.036413 -1.188685    0.201074        0.433573            1
..        ...         ...       ...         ...             ...          ...
92   0.943020    1.036413 -1.188685   -0.880256        0.396979            0
93   0.899809   -0.115288 -1.188685    1.165503       -0.444687            1
94   0.726967    0.707355  0.560019    0.551776        0.726326            0
95  -1.465965    0.213769  0.560019    1.077828       -0.444687            1
96   0.845796   -0.033024 -0.314333   -0.880256        0.433573            1

[97 rows x 6 columns]


In [ ]:
model.compile(metrics=["accuracy"])
print(model.evaluate(test_ds))
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)

2/2 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.6082
[0.0, 0.6082473993301392]
